In [1]:
import snntorch as snn
# import snntorch as surrogate
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import spikeplot as splt
from snntorch import utils
import torch.nn as nn
import torch
import torchvision

In [4]:
spike_grad = surrogate.atan()
beta = 0.5

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available else "cpu")

In [5]:
class SNN_FeedForward(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(2,12,5)
        self.pool1 = nn.MaxPool2d(2)
        self.lif1 = snn.Leaky(beta = beta, spike_grad = spike_grad, init_hidden = True)

        self.conv2 = nn.Conv2d(12,32,5)
        self.pool2 = nn.MaxPool2d(2)
        self.lif2 = snn.Leaky(beta = beta, spike_grad = spike_grad, init_hidden = True)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32*5*5, 10)
        self.lif_out = snn.Leaky(beta = beta, spike_grad = spike_grad, init_hidden = True, output = True)

    def forward(self, x_seq):
        # x_seq has shape (t, Batch_size, Polarity, x, y)
        
        spk_rec = []

        # temporal loop inthe forward pass
        for step in range(x_seq.size[0]):
            x_step = x_seq[step]

            cur1 = self.pool1(self.conv1(x_step))
            spk1 = self.lif1(cur1)

            cur2 = self.pool2(self.conv2(spk1))
            spk2 = self.lif2(cur2)

            flat = self.flatten(spk2)
            cur_out = self.fc1(flat)

            spk_out, mem_out = self.lif_out(cur_out)

            spk_rec.append(spk_out)

        return torch.stack(spk_rec, dim = 0)      

In [ ]:
class RSNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = nn.Conv2d(2,12,5)
        self.pool1 = nn.MaxPool2d(2)
        self.lif1 = snn.Leaky(beta = beta, spike_grad = spike_grad)

        self.conv2 = nn.Conv2d(12,32,5)
        self.pool2 = nn.MaxPool2d(2)
        self.lif2 = snn.Leaky(beta = beta, spike_grad = spike_grad, init_hidden = True)
        